In [2]:
import tensorflow as tf


In [4]:
import os, shutil


In [5]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [6]:
from tensorflow.keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])


In [7]:
train_dir = "D:/liqteq/Machine Learning/potatodiseaserecognition/train"
validation_dir = "D:/liqteq/Machine Learning/potatodiseaserecognition/validation"

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')

Found 1290 images belonging to 3 classes.
Found 430 images belonging to 3 classes.


In [8]:

for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20, 3)


In [10]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=50)



Epoch 1/5
100/100 [==============================] - 113s 1s/step - loss: 0.7881 - acc: 0.6379 - val_loss: 0.6186 - val_acc: 0.7500
Epoch 2/5
100/100 [==============================] - 118s 1s/step - loss: 0.4889 - acc: 0.8065 - val_loss: 0.4201 - val_acc: 0.8112
Epoch 3/5
100/100 [==============================] - 114s 1s/step - loss: 0.3221 - acc: 0.8784 - val_loss: 0.2959 - val_acc: 0.8796
Epoch 4/5
100/100 [==============================] - 128s 1s/step - loss: 0.2293 - acc: 0.9126 - val_loss: 0.2274 - val_acc: 0.9143
Epoch 5/5
100/100 [==============================] - 121s 1s/step - loss: 0.1957 - acc: 0.9212 - val_loss: 0.1699 - val_acc: 0.9347


In [11]:

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [12]:
model.save('potatoDisease.h5')

In [20]:
from tensorflow.keras.preprocessing import image
import numpy as np


def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [21]:
img_path = 'D:/liqteq/Machine Learning/potatodiseaserecognition/test/potato_healthy/774875d3-438a-4305-afe2-7d59e2925dc4___RS_HL 1759.jpg'    # dog


In [22]:
new_image = load_image(img_path)


In [23]:
pred = model.predict(new_image)


In [44]:
import operator
index, value = max(enumerate(pred[0]), key=operator.itemgetter(1))


In [45]:
if index == 0:
    print('Potato Early Blight')
elif index == 1:
    print('Potato Healthy')
else:
    print('Potato Late Blight')


Potato Healthy


In [43]:
value

0.63216347